In [3]:
#解析文件，提取url信息
import re
import csv
import requests
import urllib3
from concurrent.futures import ThreadPoolExecutor

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)



# 从csv文件中读取url列表
url_list = []
with open('./data/url_list.csv', 'r') as f:
    for line in f.readlines():
        url_list.append(line.strip())  # strip()方法是去除字符串第一个和最后一个字符的部分(默认为空格)
# print(url_list)  # 打印 url_list 列表到控制台中(可以在你的程序中以


def check_availability(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
    }

    try:
        response = requests.get(url, verify=False, headers=headers ,timeout=10)
        if response.status_code == 200:
            return (url, '可用')
        else:
            return (url, '不可用')
    except Exception as e:
        print(f'Error checking website {url}: {e}')
        return (url, '不可用')

availabe = []
inavailable = []
with ThreadPoolExecutor(max_workers=100) as executor:
    futures = [executor.submit(check_availability, url) for url in url_list]
    for future in futures:
        result = future.result()
        if result[1] == '可用':
            availabe.append(result)
        else:
            inavailable.append(result)
    # 保存available到文件csv中
    with open('./data/available.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        # writer.writerow(['URL', '状态'])
        for i in availabe:
            writer.writerow(i)
    # 保存inavailable到文件csv中
    with open('./data/inavailable.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['URL', '状态'])
        for i in inavailable:
            writer.writerow(i)

print("Done.")

Error checking website URL: Invalid URL 'URL': No schema supplied. Perhaps you meant http://URL?
Error checking website https://chat.topsearcher.top/: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))
Error checking website https://www.aicodehelper.com/: HTTPSConnectionPool(host='www.aicodehelper.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000028704D385B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error checking website https://geekch.art/: HTTPSConnectionPool(host='geekch.art', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000287040DCF10>, 'Connection to geekch.art timed out. (connect timeout=10)'))Error checking website https://poe.com/: HTTPSConnectionPool(host='poe.com', port=443): Max retries exceeded with url: / (Caused by ConnectTimeoutError(